In [84]:
import os
import sys
import json
import pandas as pd
from datetime import datetime

In [2]:
# sys.path.insert(0, os.path.abspath("../../" + 'src/scraping'))
# sys.path.insert(0, os.path.abspath("../../" + "src/modeling"))
# import basic_stats as basic

In [87]:
df = pd.read_csv("../../data/test/fortnite/video_data/fortnite_full_metadata.csv")
basic_stats_df = pd.read_csv("../../data/test/fortnite/video_data/basic_stats_04_27_20.csv")

In [88]:
basic_stats_df.set_index("videoId")
df.set_index("videoId")
df = df[df['channelId'].notnull()]

In [89]:
# for vid_id in basic_stats_df['videoId'].values:
#     if vid_id not in df['videoId'].values:
#         print(vid_id)

In [90]:
merged_df = df.merge(basic_stats_df,how="left",on="videoId")

In [91]:
merged_df.columns

Index(['categoryId', 'channelId', 'channelTitle', 'commentCount', 'date',
       'defaultLanguage', 'description', 'dislikeCount', 'duration',
       'favoriteCount', 'likeCount', 'tags', 'thumbnails', 'title', 'videoId',
       'viewCount', 'z_comments', 'z_dislikes', 'z_likes', 'z_views',
       'position', 'thumbnailFilename', 'size', 'num_rgb', 'unique_rgb_ratio',
       'mean_hue', 'mean_saturation', 'mean_brightness', 'contrast',
       'edge_score'],
      dtype='object')

In [85]:
datetime.now().strftime("_%m_%d_%y")

'_04_27_20'

In [86]:
os.path.exists(None)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType